In [ ]:
import pandas as pd
import requests

odata_url = "https://ghoapi.azureedge.net/api/"
full_url = odata_url

response = requests.get(full_url)
response.raise_for_status()

data = response.json()

df = pd.DataFrame(data['value'])
print(df.head())

for index, row in df.iterrows():
    resource = row.get('url')
    full_url = odata_url + resource

    response = requests.get(full_url)
    response.raise_for_status()

    data = response.json()

    df_value = pd.DataFrame(data['value'])

    df_value.to_csv('csvdata/' + row.get('name') + '.csv', sep=';', encoding='utf-8', index=False, header=True)

#   print(df_value.head())

#   try:
#       df_value['SpatialDim'].filter(like='GER')
#       plot = sns.barplot(x=df_value['TimeDim'], y=df_value['NumericValue'])
#       plot.set_xticklabels(plot.get_xticklabels(), rotation=45)

#       plt.savefig('analyticplots\PLOT' + row.get('name') + '.png')
#       plt.show()
#   except:
#       pass


In [3]:
import pandas as pd
import requests

indicator = "NCD_DIABETES_PREVALENCE_AGESTD"
full_url = f"https://ghoapi.azureedge.net/api/{indicator}"
response = requests.get(full_url)
response.raise_for_status()
data = response.json()
df_value = pd.DataFrame(data['value'])
df_value.to_csv('data/' + indicator + '.csv', sep=';', encoding='utf-8', index=False, header=True)

In [48]:
indicators = pd.DataFrame(requests.get("https://ghoapi.azureedge.net/api/Indicator").json()["value"])
indicators = indicators["IndicatorName"].to_list()
indicators[-100:]

['Uniform or tiered excise tax system applied on non-alcoholic beverage',
 'Low birth weight prevalence (%)',
 'Prevalence of need of assistive products (%)',
 'Years of life lost caused by hazards, aged 5 years and above',
 'Existence of a national target on physical inactivity',
 'Total population pushed below the $2.15 a day poverty line by household health expenditures (%, regional, global)',
 'Oral health interventions as part of health benefit packages: Rehabilitatative oral health care',
 'Compliance with smoke-free education facilities except universities law (score)',
 'Total productivity losses due to 5 oral diseases in million (US$)',
 'Total population pushed further below the $1.90 a day poverty line by household health expenditures (%, regional, global)',
 'Chronic hepatitis B (HBV) diagnosis coverage as a percentage of infected (%)',
 'Title of most recent survey of adults',
 'Existence of national radon map',
 'Substance use disorders service coverage, health workforce 

In [67]:
import pandas as pd

#df = pd.read_csv("data/data.csv", sep=";", keep_default_na=False)

indicator = "NCD_DIABETES_PREVALENCE_AGESTD"
data = requests.get(f"https://ghoapi.azureedge.net/api/{indicator}").json()
df = pd.DataFrame(data['value'])

# Get dimension types
dim_type_map = {"SpatialDim": None, "TimeDim": None, "Dim1": None, "Dim2": None, "Dim3": None, "NumericValue": "value"}
dim_value_map = {}
for dim in dim_type_map.keys():
    if dim_type_map[dim] is not None: continue
    dim_type = df[dim+"Type"].unique()[0]
    if dim_type is None or dim_type == "": continue 
    dim_values = requests.get(f"https://ghoapi.azureedge.net/api/DIMENSION/{dim_type}/DimensionValues").json()["value"]
    dim_value_map[dim_type.lower()] = {d["Code"]: d["Title"].lower() for d in dim_values}
    dim_type_map[dim] = dim_type.lower()

# Remove unused dimensions
dim_map = {k: v for k,v in dim_type_map.items() if v is not None}

# Get indicator name
indicator_name = requests.get(f"https://ghoapi.azureedge.net/api/Indicator?$filter=IndicatorCode eq '{indicator}'").json()["value"][0]["IndicatorName"]

# Filter data to specific columns and rename to their actual names
filtered_df = df[dim_map.keys()]
filtered_df = filtered_df.rename(columns=dim_map)

# Map Value codes to their textual values
for dim, value_map in dim_value_map.items():
    filtered_df[dim] = filtered_df[dim].map(value_map).fillna(filtered_df[dim])

# Save data to csv file
filtered_df.to_csv("data/filtered.csv", index=False, sep=";")